### Reading data saved in Part 1 of the Assignment

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [80]:
neighborhoods = pd.read_csv('toronto_neighborhood.csv')

In [81]:
neighborhoods.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.763573,-79.188711
1,M4A,North York,Victoria Village,43.770992,-79.216917
2,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.744734,-79.239476
4,M7A,Downtown Toronto,Queen's Park,43.711112,-79.284577


## Step 3: Neighborhood Clustering

In [82]:
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476
1,M7A,Downtown Toronto,Queen's Park,43.711112,-79.284577
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.836125,-79.205636
3,M5C,Downtown Toronto,St. James Town,43.761631,-79.520999
4,M4E,East Toronto,The Beaches,43.715383,-79.405678


In [83]:
toronto_data.shape

(39, 5)

In [84]:
toronto_data['Latitude'].dropna(axis=0, inplace= True)
toronto_data['Longitude'].dropna(axis=0, inplace= True)

In [85]:
toronto_data.shape

(39, 5)

In [86]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [87]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    if (lat is not np.nan and lng is not np.nan):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define FourSquare API Credential 

In [88]:
CLIENT_ID = 'DH01K1OPPGDSEFMMWPYKYDYB0X0GY1XKMUBHN0YSNFQWKZD4' # your Foursquare ID
CLIENT_SECRET = 'T3ESFKYUUTRAZPQGZG5IGOWKENS4QMSPVN21L353JXS55Q1H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DH01K1OPPGDSEFMMWPYKYDYB0X0GY1XKMUBHN0YSNFQWKZD4
CLIENT_SECRET:T3ESFKYUUTRAZPQGZG5IGOWKENS4QMSPVN21L353JXS55Q1H


## Explore Neighborhoods inToronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [89]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Runnig the function to explore the venues for each neighborhood.

In [90]:

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'] )

Harbourfront
Queen's Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village


#### Let's check the size of the resulting dataframe

In [91]:
print(toronto_venues.shape)
toronto_venues.head()

(480, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
1,Harbourfront,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant
2,Harbourfront,43.773136,-79.239476,Thai One On,43.774468,-79.241268,Thai Restaurant
3,Harbourfront,43.773136,-79.239476,Centennial Recreation Centre,43.774593,-79.236500,Athletics & Sports
4,Harbourfront,43.773136,-79.239476,TD Canada Trust,43.774952,-79.241343,Bank


Checking number of venues per neighborhood

In [92]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",91,91,91,91,91,91
Berczy Park,31,31,31,31,31,31
"Brockton,Exhibition Place,Parkdale Village",4,4,4,4,4,4
Central Bay Street,56,56,56,56,56,56
Christie,86,86,86,86,86,86
"Design Exchange,Toronto Dominion Centre",1,1,1,1,1,1
"Dovercourt Village,Dufferin",100,100,100,100,100,100
Harbourfront,8,8,8,8,8,8
"Harbourfront East,Toronto Islands,Union Station",24,24,24,24,24,24


#### the diffirent venue categories

In [93]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 144 uniques categories.


<a id='item3'></a>

##  Analyze Each Neighborhood

In [94]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,Comfort Food Restaurant,Comic Shop,Concert Hall,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hobby Shop,Hospital,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Pharmacy,Pizza Place,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [95]:
toronto_onehot.shape

(480, 145)

#### calculating the mean of the frequency of occurrence of each category

In [96]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,Comfort Food Restaurant,Comic Shop,Concert Hall,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hobby Shop,Hospital,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Pharmacy,Pizza Place,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.010989,0.000000,0.000,0.000000,0.010989,0.032967,0.000,0.043956,0.000000,0.000000,0.010989,0.010989,0.010989,0.000000,0.000000,0.00,0.010989,0.010989,0.010989,0.00,0.021978,0.010989,0.0,0.0,0.000000,0.054945,0.000000,0.010989,0.010989,0.043956,0.000000,0.021978,0.043956,0.00000,0.00000,0.021978,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.021978,0.010989,0.000000,0.000000,0.010989,0.021978,0.043956,0.000000,0.010989,0.000000,0.010989,0.000000,0.010989,0.010989,0.000000,0.010989,0.000000,0.000000,0.000000,0.010989,0.010989,0.000,0.000000,0.00000,0.00,0.00,0.010989,0.000000,0.021978,0.000000,0.000000,0.000,0.00000,0.010989,0.000000,0.010989,0.010989,0.000000,0.000000,0.0,0.000000,0.010989,0.010989,0.000000,0.000000,0.000000,0.010989,0.00,0.010989,0.00,0.000000,0.032967,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.021978,0.000000,0.00,0.010989,0.010989,0.010989,0.010989,0.00,0.000000,0.010989,0.00,0.010989,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,0.010989,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021978,0.000000,0.00000,0.000000,0.00,0.054945,0.054945,0.010989,0.00000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.032258,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.032258,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.064516,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.096774,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00000,0.00,0.00,0.032258,0.032258,0.000000,0.064516,0.000000,0.000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00

### the new size

In [97]:
toronto_grouped.shape

(14, 145)

#### Printing each neighborhood along with the top 5 most common venues

In [98]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    #print(temp)
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    #print(temp)
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                           venue  freq
0                           Café  0.05
1          Vietnamese Restaurant  0.05
2  Vegetarian / Vegan Restaurant  0.05
3                            Bar  0.04
4             Chinese Restaurant  0.04


----Berczy Park----
                venue  freq
0        Dessert Shop  0.10
1      Sandwich Place  0.10
2  Italian Restaurant  0.06
3         Pizza Place  0.06
4                 Gym  0.06


----Brockton,Exhibition Place,Parkdale Village----
                 venue  freq
0    Mobile Phone Shop  0.25
1                 Park  0.25
2          Pizza Place  0.25
3       Sandwich Place  0.25
4  American Restaurant  0.00


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2  Farmers Market  0.04
3     Cheese Shop  0.04
4            Café  0.04


----Christie----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.07
2  Italian Restaurant  0.05
3        Burge

#### Save results into a *pandas* dataframe

In [99]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [108]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Bakery,Noodle House
1,Berczy Park,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Pizza Place,Italian Restaurant,Gym,Coffee Shop,Thai Restaurant,Seafood Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Sandwich Place,Park,Pizza Place,Mobile Phone Shop,Yoga Studio,Department Store,Doner Restaurant,Discount Store,Diner,Dim Sum Restaurant
3,Central Bay Street,Coffee Shop,Cocktail Bar,Bakery,Café,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Liquor Store
4,Christie,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Juice Bar,Dessert Shop,Gym / Fitness Center


<a id='item4'></a>

## Cluster Neighborhoods

In [109]:
toronto_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,Comfort Food Restaurant,Comic Shop,Concert Hall,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hobby Shop,Hospital,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Pharmacy,Pizza Place,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.010989,0.000000,0.0,0.000000,0.010989,0.032967,0.0,0.043956,0.000000,0.000000,0.010989,0.010989,0.010989,0.000000,0.000000,0.0,0.010989,0.010989,0.010989,0.0,0.021978,0.010989,0.0,0.0,0.000000,0.054945,0.0,0.010989,0.010989,0.043956,0.000000,0.021978,0.043956,0.0,0.0,0.021978,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.021978,0.010989,0.000000,0.000000,0.010989,0.021978,0.043956,0.000000,0.010989,0.000000,0.010989,0.0,0.010989,0.010989,0.0,0.010989,0.000000,0.000000,0.000000,0.010989,0.010989,0.0,0.000000,0.0,0.0,0.0,0.010989,0.000000,0.021978,0.000000,0.000000,0.0,0.0,0.010989,0.000000,0.010989,0.010989,0.000000,0.000000,0.0,0.000000,0.010989,0.010989,0.000000,0.000000,0.000000,0.010989,0.0,0.010989,0.0,0.000000,0.032967,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.021978,0.000000,0.0,0.010989,0.010989,0.010989,0.010989,0.0,0.000000,0.010989,0.0,0.010989,0.010989,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.010989,0.010989,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021978,0.000000,0.0,0.000000,0.0,0.054945,0.054945,0.010989,0.0,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.032258,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.032258,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.064516,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.064516,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.096774,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.032258,0.032258,0.000000,0.064516,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.0,0.000000,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000

Run *k*-means to cluster the neighborhood into 5 clusters.

In [110]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 2, 2, 4, 2, 2, 2, 2], dtype=int32)

In [111]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Bakery,Noodle House
1,Berczy Park,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Pizza Place,Italian Restaurant,Gym,Coffee Shop,Thai Restaurant,Seafood Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Sandwich Place,Park,Pizza Place,Mobile Phone Shop,Yoga Studio,Department Store,Doner Restaurant,Discount Store,Diner,Dim Sum Restaurant
3,Central Bay Street,Coffee Shop,Cocktail Bar,Bakery,Café,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Liquor Store
4,Christie,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Juice Bar,Dessert Shop,Gym / Fitness Center


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [116]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476,2.0,Caribbean Restaurant,Athletics & Sports,Hakka Restaurant,Gas Station,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Yoga Studio,Dumpling Restaurant
1,M7A,Downtown Toronto,Queen's Park,43.711112,-79.284577,2.0,Bakery,Bus Line,Fast Food Restaurant,Park,Soccer Field,Intersection,Bus Station,Metro Station,Yoga Studio,Diner
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M5C,Downtown Toronto,St. James Town,43.761631,-79.520999,3.0,Liquor Store,Grocery Store,Gym,Gym / Fitness Center,Dim Sum Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Discount Store
4,M4E,East Toronto,The Beaches,43.715383,-79.405678,2.0,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Restaurant,Salon / Barbershop,Diner,Shoe Store,Burger Joint,Dessert Shop


In [117]:
toronto_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476,2.0,Caribbean Restaurant,Athletics & Sports,Hakka Restaurant,Gas Station,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Yoga Studio,Dumpling Restaurant
1,M7A,Downtown Toronto,Queen's Park,43.711112,-79.284577,2.0,Bakery,Bus Line,Fast Food Restaurant,Park,Soccer Field,Intersection,Bus Station,Metro Station,Yoga Studio,Diner
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M5C,Downtown Toronto,St. James Town,43.761631,-79.520999,3.0,Liquor Store,Grocery Store,Gym,Gym / Fitness Center,Dim Sum Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Discount Store
4,M4E,East Toronto,The Beaches,43.715383,-79.405678,2.0,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Restaurant,Salon / Barbershop,Diner,Shoe Store,Burger Joint,Dessert Shop
5,M5E,Downtown Toronto,Berczy Park,43.704324,-79.388790,2.0,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Pizza Place,Italian Restaurant,Gym,Coffee Shop,Thai Restaurant,Seafood Restaurant
6,M5G,Downtown Toronto,Central Bay Street,43.644771,-79.373306,2.0,Coffee Shop,Cocktail Bar,Bakery,Café,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Liquor Store
7,M6G,Downtown Toronto,Christie,43.657952,-79.387383,2.0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Juice Bar,Dessert Shop,Gym / Fitness Center
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.653206,-79.400049,2.0,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Bakery,Noodle House
9,M6H,West Toronto,"Dovercourt Village,Dufferin",43.648429,-79.382280,2.0,Coffee Shop,Café,Steakhouse,Gym,Hotel,Restaurant,Seafood Restaurant,Gastropub,Japanese Restaurant,Burger Joint


### Cleaning data from NaN values

In [133]:
toronto_merged.dropna(axis=0,inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int')

In [134]:
toronto_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476,2,Caribbean Restaurant,Athletics & Sports,Hakka Restaurant,Gas Station,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Yoga Studio,Dumpling Restaurant
1,M7A,Downtown Toronto,Queen's Park,43.711112,-79.284577,2,Bakery,Bus Line,Fast Food Restaurant,Park,Soccer Field,Intersection,Bus Station,Metro Station,Yoga Studio,Diner
3,M5C,Downtown Toronto,St. James Town,43.761631,-79.520999,3,Liquor Store,Grocery Store,Gym,Gym / Fitness Center,Dim Sum Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Discount Store
4,M4E,East Toronto,The Beaches,43.715383,-79.405678,2,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Restaurant,Salon / Barbershop,Diner,Shoe Store,Burger Joint,Dessert Shop
5,M5E,Downtown Toronto,Berczy Park,43.704324,-79.388790,2,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Pizza Place,Italian Restaurant,Gym,Coffee Shop,Thai Restaurant,Seafood Restaurant
6,M5G,Downtown Toronto,Central Bay Street,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Bakery,Café,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Liquor Store
7,M6G,Downtown Toronto,Christie,43.657952,-79.387383,2,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Juice Bar,Dessert Shop,Gym / Fitness Center
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.653206,-79.400049,2,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Bakery,Noodle House
9,M6H,West Toronto,"Dovercourt Village,Dufferin",43.648429,-79.382280,2,Coffee Shop,Café,Steakhouse,Gym,Hotel,Restaurant,Seafood Restaurant,Gastropub,Japanese Restaurant,Burger Joint
10,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.661608,-79.464763,2,Mexican Restaurant,Café,Thai Restaurant,Bar,Gastropub,Speakeasy,Park,Cajun / Creole Restaurant,Diner,Discount Store


In [123]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,"Adelaide,King,Richmond",Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Bakery,Noodle House
1,2,Berczy Park,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Pizza Place,Italian Restaurant,Gym,Coffee Shop,Thai Restaurant,Seafood Restaurant
2,0,"Brockton,Exhibition Place,Parkdale Village",Sandwich Place,Park,Pizza Place,Mobile Phone Shop,Yoga Studio,Department Store,Doner Restaurant,Discount Store,Diner,Dim Sum Restaurant
3,2,Central Bay Street,Coffee Shop,Cocktail Bar,Bakery,Café,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Liquor Store
4,2,Christie,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Juice Bar,Dessert Shop,Gym / Fitness Center


Finally, let's visualize the resulting clusters

In [124]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,West Toronto,0,Sandwich Place,Park,Pizza Place,Mobile Phone Shop,Yoga Studio,Department Store,Doner Restaurant,Discount Store,Diner,Dim Sum Restaurant


#### Cluster 2

In [128]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,East Toronto,1,Empanada Restaurant,Pizza Place,Dessert Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Discount Store,Diner,Dim Sum Restaurant,Department Store


#### Cluster 3

In [129]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Caribbean Restaurant,Athletics & Sports,Hakka Restaurant,Gas Station,Thai Restaurant,Bakery,Bank,Fried Chicken Joint,Yoga Studio,Dumpling Restaurant
1,Downtown Toronto,2,Bakery,Bus Line,Fast Food Restaurant,Park,Soccer Field,Intersection,Bus Station,Metro Station,Yoga Studio,Diner
4,East Toronto,2,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Restaurant,Salon / Barbershop,Diner,Shoe Store,Burger Joint,Dessert Shop
5,Downtown Toronto,2,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Pizza Place,Italian Restaurant,Gym,Coffee Shop,Thai Restaurant,Seafood Restaurant
6,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Bakery,Café,Farmers Market,Cheese Shop,Beer Bar,Seafood Restaurant,Steakhouse,Liquor Store
7,Downtown Toronto,2,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Japanese Restaurant,Juice Bar,Dessert Shop,Gym / Fitness Center
8,Downtown Toronto,2,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Bakery,Noodle House
9,West Toronto,2,Coffee Shop,Café,Steakhouse,Gym,Hotel,Restaurant,Seafood Restaurant,Gastropub,Japanese Restaurant,Burger Joint
10,Downtown Toronto,2,Mexican Restaurant,Café,Thai Restaurant,Bar,Gastropub,Speakeasy,Park,Cajun / Creole Restaurant,Diner,Discount Store
11,West Toronto,2,Coffee Shop,Gym,Park,Yoga Studio,Mexican Restaurant,Burrito Place,Portuguese Restaurant,Café,Chinese Restaurant,College Auditorium


#### Cluster 4

In [131]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,3,Liquor Store,Grocery Store,Gym,Gym / Fitness Center,Dim Sum Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Discount Store


#### Cluster 5

In [132]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,4,Park,Yoga Studio,Dessert Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Discount Store,Diner,Dim Sum Restaurant,Department Store
